# CHAPTER 7 Data Cleaning and Preparation

In [1]:
import pandas as pd
import numpy as np
import sys

During the course of doing data analysis and modeling, a significant amount of time
is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up 80% or more of an analyst’s time. 

## Handling Missing Data

all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data. We call this a sentinel value that
can be easily detected:

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by refer‐
ring to missing data as NA, which stands for not available. In statistics applications,
NA data may either be data that does not exist or that exists but was not observed
(through problems with data collection, for example). When cleaning up data for
analysis, it is often important to do analysis on the missing data itself to identify data
collection problems or potential biases in the data caused by missing data.

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to
do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.
On a Series, it returns the Series with only the non-null data and index values:

In [5]:
from numpy import nan as NA

In [6]:
data = pd.Series([1,NA,3.5,NA,7])

In [7]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
# that is equal to:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value:

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], 
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned = data.dropna()  # 只要有null，就删除整行
cleaned

,0,1,2
0,1.0,6.5,3.0


In [11]:
# Passing how='all' will only drop rows that are all NA:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [12]:
# To drop columns in the same way, pass axis=1:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [14]:
# A related way to filter out DataFrame rows tends to concern time series data. Suppose
# you want to keep only rows containing a certain number of observations. You can
# indicate this with the thresh argument:
df = pd.DataFrame(np.random.randn(7,3))
df

,0,1,2
0,1.506736,-0.133495,0.408375
1,0.314276,-0.262825,0.775007
2,-1.809538,1.560685,-0.004200
3,-1.942281,0.453585,-1.165535
4,-1.747193,-0.637860,2.641643
5,-2.029688,0.752997,0.288016
6,0.621540,0.428856,-1.633618


In [15]:
df.iloc[:4,1] = NA
df.iloc[:2,2] = NA
df

,0,1,2
0,1.506736,NaN,NaN
1,0.314276,NaN,NaN
2,-1.809538,NaN,-0.004200
3,-1.942281,NaN,-1.165535
4,-1.747193,-0.637860,2.641643
5,-2.029688,0.752997,0.288016
6,0.621540,0.428856,-1.633618


In [16]:
df.dropna(thresh=2)

,0,1,2
2,-1.809538,NaN,-0.004200
3,-1.942281,NaN,-1.165535
4,-1.747193,-0.637860,2.641643
5,-2.029688,0.752997,0.288016
6,0.621540,0.428856,-1.633618


In [17]:
df.dropna(thresh=3) #thresh=n是指：保留下来的每一行，其非NA的数目>=n。

,0,1,2
4,-1.747193,-0.637860,2.641643
5,-2.029688,0.752997,0.288016
6,0.621540,0.428856,-1.633618


### Filling In Missing Data

In [18]:
df.fillna(0)

,0,1,2
0,1.506736,0.000000,0.000000
1,0.314276,0.000000,0.000000
2,-1.809538,0.000000,-0.004200
3,-1.942281,0.000000,-1.165535
4,-1.747193,-0.637860,2.641643
5,-2.029688,0.752997,0.288016
6,0.621540,0.428856,-1.633618


In [19]:
# Calling fillna with a dict, you can use a different fill value for each column:
df.fillna({1:0.5,2:0})

,0,1,2
0,1.506736,0.500000,0.000000
1,0.314276,0.500000,0.000000
2,-1.809538,0.500000,-0.004200
3,-1.942281,0.500000,-1.165535
4,-1.747193,-0.637860,2.641643
5,-2.029688,0.752997,0.288016
6,0.621540,0.428856,-1.633618


In [20]:
# fillna returns a new object, but you can modify the existing object in-place:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,1.506736,0.000000,0.000000
1,0.314276,0.000000,0.000000
2,-1.809538,0.000000,-0.004200
3,-1.942281,0.000000,-1.165535
4,-1.747193,-0.637860,2.641643
5,-2.029688,0.752997,0.288016
6,0.621540,0.428856,-1.633618


In [21]:
# The same interpolation methods available for reindexing can be used with fillna:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-0.523367,-0.862837,-1.092785
1,0.177654,0.728744,1.137471
2,-2.093415,NaN,-0.084161
3,1.502386,NaN,-1.683451
4,-0.621485,NaN,NaN
5,0.478271,NaN,NaN


In [22]:
df.fillna(method='ffill')  # ffill 向下填充

,0,1,2
0,-0.523367,-0.862837,-1.092785
1,0.177654,0.728744,1.137471
2,-2.093415,0.728744,-0.084161
3,1.502386,0.728744,-1.683451
4,-0.621485,0.728744,-1.683451
5,0.478271,0.728744,-1.683451


In [23]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.523367,-0.862837,-1.092785
1,0.177654,0.728744,1.137471
2,-2.093415,0.728744,-0.084161
3,1.502386,0.728744,-1.683451
4,-0.621485,NaN,-1.683451
5,0.478271,NaN,-1.683451


In [24]:
df.fillna(df.mean())

,0,1,2
0,-0.523367,-0.862837,-1.092785
1,0.177654,0.728744,1.137471
2,-2.093415,-0.067047,-0.084161
3,1.502386,-0.067047,-1.683451
4,-0.621485,-0.067047,-0.430731
5,0.478271,-0.067047,-0.430731


In [25]:
df.fillna(df.mean(),axis=0)

,0,1,2
0,-0.523367,-0.862837,-1.092785
1,0.177654,0.728744,1.137471
2,-2.093415,-0.067047,-0.084161
3,1.502386,-0.067047,-1.683451
4,-0.621485,-0.067047,-0.430731
5,0.478271,-0.067047,-0.430731


In [26]:
# With fillna you can do lots of other things with a little creativity. For example, you
# might pass the mean or median value of a Series:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

In [27]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [28]:
# The DataFrame method duplicated returns a boolean Series indicating whether each
# row is a duplicate (has been observed in a previous row) or not:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [29]:
# Relatedly, drop_duplicates returns a DataFrame where the duplicated array is
# False:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [30]:
# Both of these methods by default consider all of the columns; alternatively, you can
# specify any subset of them to detect duplicates. Suppose we had an additional column
# of values and wanted to filter duplicates only based on the 'k1' column:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [31]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [32]:
# duplicated and drop_duplicates by default keep the first observed value combina‐
# tion. Passing keep='last' will return the last one:
data.drop_duplicates(['k1','k2'],keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [33]:
data.drop_duplicates(['k1'],keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [34]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
 'Pastrami', 'corned beef', 'Bacon',
 'pastrami', 'honey ham', 'nova lox'],
 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [35]:
# Suppose you wanted to add a column indicating the type of animal that each food
# came from. Let’s write down a mapping of each distinct meat type to the kind of
# animal:
meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}

In [36]:
# The map method on a Series accepts a function or dict-like object containing a map‐
# ping, but here we have a small problem in that some of the meats are capitalized and
# others are not. Thus, we need to convert each value to lowercase using the str.lower
# Series method:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [37]:
lowercased.map(meat_to_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [38]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### Replacing ValuesReplacing Values

In [39]:
# Filling in missing data with the fillna method is a special case of more general value
# replacement. As you’ve already seen, map can be used to modify a subset of values in
# an object but replace provides a simpler and more flexible way to do so. Let’s con‐
# sider this Series:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [40]:
# The -999 values might be sentinel values for missing data. To replace these with NA
# values that pandas understands, we can use replace, producing a new Series (unless
# you pass inplace=True):
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [41]:
# If you want to replace multiple values at once, you instead pass a list and then the
# substitute value:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [42]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [43]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [44]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
 index=['Ohio', 'Colorado', 'New York'],
 columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [56]:
# Like a Series, the axis indexes have a map method:
transform = lambda x: x[:4].upper()
transform

<function __main__.<lambda>(x)>

In [57]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [58]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [60]:
# If you want to create a transformed version of a dataset without modifying the origi‐
# nal, a useful method is rename:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [61]:
# Notably, rename can be used in conjunction with a dict-like object providing new val‐
# ues for a subset of the axis labels:
data.rename(index={'OHIO':'INDIANA'},
           columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [63]:
# rename saves you from the chore of copying the DataFrame manually and assigning
# to its index and columns attributes. Should you wish to modify a dataset in-place,
# pass inplace=True:
data.rename(index={'OHIO':'INDIANA'},inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [64]:
# Continuous data is often discretized or otherwise separated into “bins” for analysis.
# Suppose you have data about a group of people in a study, and you want to group
# them into discrete age buckets:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [65]:
# Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
# do so, you have to use cut, a function in pandas:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [66]:
# The object pandas returns is a special Categorical object. The output you see
# describes the bins computed by pandas.cut. You can treat it like an array of strings
# indicating the bin name; internally it contains a categories array specifying the dis‐
# tinct category names along with a labeling for the ages data in the codes attribute:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [67]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [68]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [70]:
# Consistent with mathematical notation for intervals, a parenthesis means that the side
# is open, while the square bracket means it is closed (inclusive). You can change which
# side is closed by passing right=False:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)  #右边不包含

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [71]:
# You can also pass your own bin names by passing a list or array to the labels option:
group_names = ['Youth','YoungAdult','MiddleAged','Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [77]:
# If you pass an integer number of bins to cut instead of explicit bin edges, it will com‐
# pute equal-length bins based on the minimum and maximum values in the data.
# Consider the case of some uniformly distributed data chopped into fourths:
data = np.random.rand(20)
print(data,'\n')
pd.cut(data,4,precision=2)  # The precision=2 option limits the decimal precision to two digits.

[0.13406409 0.38331261 0.00499024 0.3698894  0.4624845  0.59059517
 0.87602293 0.72230084 0.46870946 0.40726337 0.59114254 0.20591751
 0.21244888 0.12589287 0.62676083 0.60241297 0.13074857 0.88125538
 0.86709096 0.42377409] 



[(0.0041, 0.22], (0.22, 0.44], (0.0041, 0.22], (0.22, 0.44], (0.44, 0.66], ..., (0.44, 0.66], (0.0041, 0.22], (0.66, 0.88], (0.66, 0.88], (0.22, 0.44]]
Length: 20
Categories (4, interval[float64]): [(0.0041, 0.22] < (0.22, 0.44] < (0.44, 0.66] < (0.66, 0.88]]

In [78]:
# A closely related function, qcut, bins the data based on sample quantiles. Depending
# on the distribution of the data, using cut will not usually result in each bin having the
# same number of data points. Since qcut uses sample quantiles instead, by definition
# you will obtain roughly equal-size bins:
data = np.random.randn(1000) # Normally distributed

cats = pd.cut(data,4) 
cats

[(-0.492, 1.192], (-0.492, 1.192], (-0.492, 1.192], (-0.492, 1.192], (-0.492, 1.192], ..., (-0.492, 1.192], (-0.492, 1.192], (-0.492, 1.192], (-2.175, -0.492], (-0.492, 1.192]]
Length: 1000
Categories (4, interval[float64]): [(-3.865, -2.175] < (-2.175, -0.492] < (-0.492, 1.192] < (1.192, 2.875]]

In [79]:
pd.value_counts(cats)

(-0.492, 1.192]     567
(-2.175, -0.492]    309
(1.192, 2.875]      109
(-3.865, -2.175]     15
dtype: int64

In [82]:
cats = pd.qcut(data,4)  # Cut into quartiles
cats 

[(-0.732, -0.0239], (0.688, 2.875], (-0.732, -0.0239], (-0.0239, 0.688], (0.688, 2.875], ..., (-0.732, -0.0239], (0.688, 2.875], (0.688, 2.875], (-3.859, -0.732], (0.688, 2.875]]
Length: 1000
Categories (4, interval[float64]): [(-3.859, -0.732] < (-0.732, -0.0239] < (-0.0239, 0.688] < (0.688, 2.875]]

In [84]:
pd.value_counts(cats)  # euqal quartiles

(-3.859, -0.732]     250
(-0.732, -0.0239]    250
(-0.0239, 0.688]     250
(0.688, 2.875]       250
dtype: int64

In [85]:
# Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive):
pd.qcut(data,[0, 0.1, 0.5, 0.9, 1.])

[(-1.325, -0.0239], (-0.0239, 1.246], (-1.325, -0.0239], (-0.0239, 1.246], (-0.0239, 1.246], ..., (-1.325, -0.0239], (-0.0239, 1.246], (-0.0239, 1.246], (-3.859, -1.325], (-0.0239, 1.246]]
Length: 1000
Categories (4, interval[float64]): [(-3.859, -1.325] < (-1.325, -0.0239] < (-0.0239, 1.246] < (1.246, 2.875]]

### Detecting and Filtering Outliers